In [ ]:
#Guardar l'arxiu
sales.write.format("delta").save("/path/to/delta-table")

Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [1]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

NameError: name 'configure_spark_with_delta_pip' is not defined

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [ ]:
#Arxiu Parquet 
parquetFileDF = spark.read.parquet("C:\\Users\\paula\\Documents\\IA\\Quatri_6\\BDA\\BDA-P1\\datalake\\sales_data\\2024-04-17_SuperstoreSalesTraining.parquet")
parquetFileDF.show()


In [ ]:
sales.write.format("delta").saveAsTable("sales")

In [ ]:
DF = spark.sql("""SELECT salary, avg(cast(last_evaluation as float)) FROM HR GROUP BY salary""")
DF.show()